# Обработка запросов


# Обработка запросов 
- Обработка запросов - набор активностей, используемых при заборе данных из БД.
- Базовые шаги
    1. Парсинг и трансляция
    2. Оптимизация
    3. Вычисление 

До того, как будет обработан запрос, система должна транслировать запрос в подходящую форму. Внутренне представление может различаться (обычно для общего случая рассматривается реляционная алгебра)

# Обработка запросов (схема) 

# Приведение к реляционной алгебре
Рассмотрим запрос:
```sql
SELECT salary
  FROM students 
 WHERE salary > 20000
```

Он может быть переписан на язык реляционной алгебры 

- $ \sigma_{salary>20000}(\Pi_{salary}(students)) $
- $ \Pi_{salary}(\sigma_{salary>20000}(students)) $


Более того, для каждой операции реляционной алегбры может быть выбран один из нескольких разных алгоритмов. 

В зависимости от структуры хранения и вариантов обработки данных может быть осуществлена различная выборка.

Для спецификации, каким образом будет происходить механизм обработки, необходимо не только перевести запрос
в выражение реляционной алгебры, но также и алгоритм вычисления каждой операции реляционный алгебры, причем с 
учетом количества данных и их структуры хранения. 

Операция реляционной алгебры со связанной с ней инструкцией по ее вычислению называется __примитив вычисления__ (evaluation primitive)

Последовательность примитивов вычисления, которые могут быть использованы для вычисления запроса называются __планом выполнения запроса__ (query execution plan). Механизм вычисления запроса (query execution engine) принимает на вход план выполнения запроса, выполняет его, и возвращает результат к запросу. 

Различные планы выполнения для одного и того же запроса могут иметь различную стоимость. От пользователя не ожидается, что он должен писать запросы с наиболее эффективным планом. Это задача СУБД выбрать такой план запроса, который будет минимальный по стоимости для запроса. Данная задача называется оптимизацией запросов (query optimization).

На данный момент сосредоточимся на варианте с реляционной алгеброй, другие варианты будут рассмотрены для конкретных
СУБД. 

# Меры оценки запросов


Стоимости меры оценки запросов может быть измерена в терминах различных ресурсов:

- I/O. Доступ к физическому диску 
- Время ЦП на обработку 
- Стоимость коммуникации (в параллельных и распределенных СУБД)


Для СУБД больших размеров обычно основным является вопрос I/O на дисках, однако, с учетом активного развития SSD и
In-Memory баз данных иногда возникает сильное влияние и других факторов, поэтому включают и другие значения. 

Для простоты изначально не будем включать затраты ЦП для вычисления стоимости, однако, необходимо иметь в виду, что 
в реальных СУБД учитывается гораздо больше факторов. 

Оценка запроса:

- количество переданных блоков. $ t_T$
- количество I/O произвольного доступа $t_s$ 

Примерные значения для HDD $t_s = 4$ миллисекунды и $t_T = 0.1$ миллисекунды в предположении, что размер блока - 4KB, а скорость передачи 40МБ/с. Для SSD $ t_s = 90 $ микросекунд и $ t_T $ = 4 микросекунды для 4KB блока.


Для данных, которые уже в ОП, чтение выполянет в единицах значения кеша. В предположении считывания 
всего блока $t_T < 1$ микросекунды. $ t_S < 100$ наносекунд. 

В современных СУБД значения различных базовых элементов поиска настраиваются либо автоматически при установке СУБД, либо исправляются в конфигурационном файле.

## Чтение блоков vs. Запись блоков

Обычно запись блока более чем в 2 раза дороже, по сравнению с чтением блока на магнитный диск, так как после записи происходит еще чтение блока для проверки корректности записи. 

Практически везде в дальнейших примерах запись итогового результат на диск не будет учитываться. 

## Буфер обмена 

Стоимость любого алгоритма, который будет рассматриваться далее, зависит от результата размере буфера в ОП. Лучший случай - данные всегда считывются из буфера, худший - в буфере совсем немного блоков. 

В современных СУБД худший случай является слишком пессимистичным. Обычно при расчете стоимости используется число M - количество памяти, доступное оператору, как параметр. В PostgresQL количество памяти, доступное для запроса (effective cache size) - 4 ГБ, для целей оптимизации стоимости. 

Также мы предполагаем, что данные всегда считываются с диска. Обычно довольно большое число блоков нахрдится в буфере ОП. Для учета такой возможности различные СУБД используют свои допущения: например, PostgreSQL рассматривает стоимость чтения случайной страницы как $1/10^{ой}$ от чтения страницы на диске, для моделирования ситуации о том, что 90% страниц находятся уже в буфере.

Для $B^{+}$ деревьев часто предполагают, что все внутренние вершины находятся в памяти